Mount your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
# MODIFY HERE TO YOUR PATH IN GOOGLE DRIVE
%cd /content/gdrive/MyDrive/CryptoML

---

Execute the functions to train the model

In [ ]:
import speck as sp
import numpy as np
from pickle import dump
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2

bs = 5000;
dir = './models/'

def cyclic_lr(num_epochs, high_lr, low_lr):
  '''
  This function is to get the learning rate according to the number of epochs
  and ensure the learning rate is within the range.

  Inputs:
    - num_epochs: Number of epochs
    - high_lr: Highest learning rate
    - low_lr: Lowest learning rate

  Output:
  The learning rate of the corresponding number of epochs will be returned

  '''
  res = lambda i: low_lr + ((num_epochs-1) - i % num_epochs)/(num_epochs-1) * (high_lr - low_lr)
  return res


def make_checkpoint(datei):
  '''
  This function makes a checkpoint of the model. 

  Input: datai is the directory where we wish to save the model
  Output: A saved Keras model when the validation loss is at its least

  '''
  assert datei[:9] == "./models/"
  assert datei[-3:] == ".h5"
    
  res = ModelCheckpoint(datei, monitor='val_loss', save_best_only = True)
  return res


def make_resnet(num_blocks=2, num_filters=32, num_outputs=1, d1=64, d2=64, word_size=16, ks=3, depth=5, reg_param=0.0001, final_activation='sigmoid'):
  '''
  This function makes residual tower of convolutional blocks.
  
  Inputs:
    - num_blocks: Number of plaintexts
    - num_filters: Number of filters in the Conv1D layers
    - num_outputs: Number of outputs for the final prediction (0 or 1 in this case)
    - d1: Units in the first Dense layer 
    - d2: Units in the second Dense layer
    - word_size: Word size which will affect the shape of Input layer
    - ks: Kernel size for the Conv1D layers in the residual blocks
    - depth: The number of residual blocks
    - reg_param: Parameter for regularizers
    - final_activation: The activation function in the final Dense layer

  Output: 
  A Keras model

  '''
  assert num_outputs == 1
  assert num_blocks in [2,4]

  # Input and preprocessing layers
  inp = Input(shape=(num_blocks * word_size * 2))
  rs = Reshape((2 * num_blocks, word_size))(inp)
  perm = Permute((2,1))(rs)
  
  # Add a single residual layer that will expand the data to num_filters channels
  # This is a bit-sliced layer
  conv0 = Conv1D(num_filters, kernel_size=1, padding='same', kernel_regularizer=l2(reg_param))(perm)
  conv0 = BatchNormalization()(conv0)
  conv0 = Activation('relu')(conv0)

  # Add residual blocks
  shortcut = conv0
  for _ in range(depth):
    conv1 = Conv1D(num_filters, kernel_size=ks, padding='same', kernel_regularizer=l2(reg_param))(shortcut)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv2 = Conv1D(num_filters, kernel_size=ks, padding='same',kernel_regularizer=l2(reg_param))(conv1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    shortcut = Add()([shortcut, conv2])

  # Add prediction head
  flat1 = Flatten()(shortcut)
  dense1 = Dense(d1,kernel_regularizer=l2(reg_param))(flat1)
  dense1 = BatchNormalization()(dense1)
  dense1 = Activation('relu')(dense1)
  dense2 = Dense(d2, kernel_regularizer=l2(reg_param))(dense1)
  dense2 = BatchNormalization()(dense2)
  dense2 = Activation('relu')(dense2)
  out = Dense(num_outputs, activation=final_activation, kernel_regularizer=l2(reg_param))(dense2)
  model = Model(inputs=inp, outputs=out)

  return model


def train_speck_distinguisher(num_epochs, num_rounds=7, depth=1, num_blocks=2, diffa=(0x0040,0), diffb=(0x0020,0)):
  '''
  This function trains the model and saves it to the models folder.

  Inputs:
    - num_epochs: Number of epochs you want to train
    - num_rounds: Number of rounds of the Speck
    - depth: The number of residual blocks
    - num_blocks: Number of plaintexts
    - diffa: Bit difference 1
    - diffb: Bit difference 2 (if training with 4 ciphertexts)

  Outputs:
    - net: Network
    - h: The trained model
    
  '''
  assert num_epochs > 0
  assert num_rounds > 0
  assert depth > 0
  assert num_blocks in [2, 4]
  assert type(diffa) is tuple and type(diffb) is tuple

  # Create the network
  net = make_resnet(num_blocks=num_blocks, depth=depth, reg_param=10**-5)
  net.compile(optimizer='adam',loss='mse',metrics=['acc'])

  # Generate training and validation data
  if num_blocks == 2:
    X, Y = sp.make_train_data_2pt(10**7, num_rounds, diff=diffa)
    X_eval, Y_eval = sp.make_train_data_2pt(10**6, num_rounds, diff=diffa)
  
  else:
    X, Y = sp.make_train_data_4pt(10**7, num_rounds, diffa=diffa, diffb=diffb)
    X_eval, Y_eval = sp.make_train_data_4pt(10**6, num_rounds, diffa=diffa, diffb=diffb)

  # Set up model checkpoint
  check = make_checkpoint(dir+'best'+str(num_rounds)+'depth'+str(depth)+'.h5')

  # Create learnrate schedule
  lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001))

  # Train and evaluate
  h = net.fit(X,Y,epochs=num_epochs,batch_size=bs,validation_data=(X_eval, Y_eval), callbacks=[lr,check])
  np.save(dir+'h'+str(num_rounds)+'r_depth'+str(depth)+'.npy', h.history['val_acc'])
  np.save(dir+'h'+str(num_rounds)+'r_depth'+str(depth)+'.npy', h.history['val_loss'])
  dump(h.history,open(dir+'hist'+str(num_rounds)+'r_depth'+str(depth)+'.p','wb'))
  print("Best validation accuracy: ", np.max(h.history['val_acc']))

  return net, h

---

Modify the parameters and run this cell to start training the model

In [ ]:
'''
Parameter description: 
  - num_epochs: number of epochs, 
  - num_rounds: Number of rounds of the Speck
  - depth: The number of residual blocks
  - num_blocks: Number of plaintexts
  - diffa: Bit difference 1
  - diffb: Bit difference 2 (if training with 4 ciphertexts)

'''

# MODIFY HERE:                       ↓             ↓        ↓             ↓            ↓                  ↓
train_speck_distinguisher(num_epochs=1, num_rounds=5, depth=1, num_blocks=4, diffa=(0x0040,0), diffb=(0x0020,0))